In [32]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import sys, six

# Import Data Sccience libraries
import pandas as pd

# Instantiates a client
client = language.LanguageServiceClient()

In [5]:
small_suicide_df = pd.DataFrame.from_csv("sample_suicide_watch_text.csv",encoding = "ISO-8859-1").iloc[0:10,:]

small_suicide_df

In [34]:
def sentiment_text(text):
    """Detects sentiment in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects sentiment in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    sentiment = client.analyze_sentiment(document).document_sentiment
    
    return [sentiment.score,sentiment.magnitude]

In [31]:
def entity_sentiment_text(text):
    """Detects entity sentiment in the provided text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    # Detect and send native Python encoding to receive correct word offsets.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    for entity in result.entities:
        print('Mentions: ')
        print(u'Name: "{}"'.format(entity.name))
        for mention in entity.mentions:
            print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
            print(u'  Content : {}'.format(mention.text.content))
            print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
            print(u'  Sentiment : {}'.format(mention.sentiment.score))
            print(u'  Type : {}'.format(mention.type))
        print(u'Salience: {}'.format(entity.salience))
        print(u'Sentiment: {}\n'.format(entity.sentiment))

In [29]:
entity_sentiment_text(record)

Mentions: 
Name: "words"
  Begin Offset : 57
  Content : words
  Magnitude : 0.0
  Sentiment : 0.0
  Type : 2
Salience: 0.3393266499042511
Sentiment: 

Mentions: 
Name: "misconception"
  Begin Offset : 199
  Content : misconception
  Magnitude : 0.800000011920929
  Sentiment : -0.800000011920929
  Type : 2
Salience: 0.19303403794765472
Sentiment: magnitude: 1.600000023841858
score: -0.800000011920929


Mentions: 
Name: "anyone"
  Begin Offset : 330
  Content : anyone
  Magnitude : 0.30000001192092896
  Sentiment : -0.30000001192092896
  Type : 2
Salience: 0.09136639535427094
Sentiment: magnitude: 0.699999988079071
score: -0.30000001192092896


Mentions: 
Name: "attention"
  Begin Offset : 144
  Content : attention
  Magnitude : 0.0
  Sentiment : 0.0
  Type : 2
Salience: 0.09088879823684692
Sentiment: 

Mentions: 
Name: "pain"
  Begin Offset : 218
  Content : pain
  Magnitude : 0.20000000298023224
  Sentiment : -0.20000000298023224
  Type : 2
Salience: 0.05292052775621414
Sentiment: mag

In [59]:
extra_data = []

for record in small_suicide_df.iterrows():
    text = record[1]["text"]
    
    extra_record = sentiment_text(text)
    extra_data.append(extra_record)

In [72]:
extra_data_df = pd.DataFrame(extra_data, columns=["Score","Magnitude"],index=small_suicide_df.index)

extra_data_df

In [70]:
small_combined_df = small_suicide_df.join(extra_data_df)

small_combined_df

In [108]:
depression_df = pd.DataFrame.from_csv("depressionTrainingSet3.csv")

In [119]:
dep500_df = depression_df.iloc[0:500,0:1].reset_index()

In [127]:
dep500_extra = []

for record in dep500_df.iterrows():
    text = record[1]["text"]
    
    extra_record = sentiment_text(text)
    dep500_extra.append(extra_record)

In [131]:
dep500_extra_df = pd.DataFrame(dep500_extra,columns=["sentiment","magnitude"])

dep500_extra_df.shape

(500, 2)

In [135]:
big_dep500_df = dep500_df.join(dep500_extra_df)

In [137]:
df = big_dep500_df

In [139]:
# Start doing classification with SKlearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

In [144]:
X = df.loc[:,["sentiment","magnitude"]]
y = df.loc[:,"risk_of_suicide"]

In [148]:
# importing the non-suicide data


(500,)